# Agent_Graph

In [ ]:
"""
AGENT_GRAPH.PY — CORE ORCHESTRATION FILE
=======================================

This is the MAIN file of the project.
This is the only file you actually RUN from the terminal.

Everything else (job discovery, LLM reasoning, email sending)
exists to SUPPORT this file.

Think of this file as:
- the brain
- the workflow controller
- the traffic police for the agent

If you understand this file, you understand the project.
"""

# LangGraph is used to build agent workflows as graphs instead of chains
from langgraph.graph import StateGraph, END

# TypedDict is used to clearly define the agent's shared memory (state)
from typing import TypedDict, List, Dict, Any

# External module that fetches raw job postings
from job_discovery import get_jobs


# -------------------------------------------------------------------
# AGENT STATE (SHARED MEMORY)
# -------------------------------------------------------------------
"""
AgentState defines EVERYTHING the agent knows at any point in time.

LangGraph forces us to be explicit about state.
This avoids hidden variables and makes debugging much easier.

Every node:
- receives this state
- modifies it
- returns it
"""

class AgentState(TypedDict):
    jobs: List[Dict[str, Any]]           # jobs currently under consideration
    skipped_jobs: List[Dict[str, Any]]   # jobs rejected + reason
    roles_targeted: List[str]            # user preferences
    max_experience_allowed: int          # user constraint
    decisions: List[Dict[str, Any]]      # final agent decisions


# -------------------------------------------------------------------
# CREATE THE GRAPH
# -------------------------------------------------------------------
"""
StateGraph is LangGraph's main abstraction.
We tell it what the shared state looks like.
"""

graph = StateGraph(AgentState)


# -------------------------------------------------------------------
# NODE 1 — JOB DISCOVERY
# -------------------------------------------------------------------
"""
This node is responsible for bringing EXTERNAL DATA into the agent.

Important design choice:
- The agent does NOT scrape websites itself.
- It receives already-structured job data from job_discovery.py

This keeps responsibilities clean.
"""

def discover_jobs_node(state: AgentState) -> AgentState:
    result = get_jobs()   # expected to return a dictionary

    # Inject discovered jobs into agent memory
    state["jobs"] = result["jobs"]

    # Jobs skipped during scraping phase (duplicates, broken posts, etc.)
    state["skipped_jobs"] = result["skipped_jobs"]

    return state


# -------------------------------------------------------------------
# NODE 2 — MASKED COMPANY FILTER
# -------------------------------------------------------------------
"""
Some job postings hide company names using '*'.
These are often spammy or low quality.

Instead of deleting them:
- we skip them
- we STORE the reason

This helps with transparency and reporting.
"""

def check_masked_node(state: AgentState) -> AgentState:
    clean_jobs = []
    skipped = state["skipped_jobs"]

    for job in state["jobs"]:
        if "*" in job["title"] or "*" in job["company"]:
            skipped.append({
                "title": job["title"],
                "company": job["company"],
                "reason": "masked_company"
            })
        else:
            clean_jobs.append(job)

    state["jobs"] = clean_jobs
    state["skipped_jobs"] = skipped
    return state


# -------------------------------------------------------------------
# NODE 3 — ROLE MATCHING
# -------------------------------------------------------------------
"""
Before using LLMs, we apply cheap rule-based filters.

Why?
- Faster
- Cheaper
- Avoids unnecessary LLM calls

Here we check whether the job title matches
the roles defined in user_profile.json.
"""

def check_role_node(state: AgentState) -> AgentState:
    clean_jobs = []
    skipped = state["skipped_jobs"]

    roles = [r.lower() for r in state["roles_targeted"]]

    for job in state["jobs"]:
        title = job["title"].lower()

        if any(role in title for role in roles):
            clean_jobs.append(job)
        else:
            skipped.append({
                "title": job["title"],
                "company": job["company"],
                "reason": "role_mismatch"
            })

    state["jobs"] = clean_jobs
    state["skipped_jobs"] = skipped
    return state


# -------------------------------------------------------------------
# NODE 4 — EXPERIENCE FILTER
# -------------------------------------------------------------------
"""
This node enforces experience constraints.

Applying for roles far above experience level
is usually a waste of time.

We filter early to protect the user.
"""

def check_experience_node(state: AgentState) -> AgentState:
    clean_jobs = []
    skipped = state["skipped_jobs"]
    max_exp = state["max_experience_allowed"]

    for job in state["jobs"]:
        required = job.get("required_experience")

        if required is not None and required > max_exp:
            skipped.append({
                "title": job["title"],
                "company": job["company"],
                "required_experience": required,
                "reason": "experience_too_high"
            })
        else:
            clean_jobs.append(job)

    state["jobs"] = clean_jobs
    state["skipped_jobs"] = skipped
    return state


# -------------------------------------------------------------------
# NODE 5 — FINAL DECISION
# -------------------------------------------------------------------
"""
At this stage, only high-quality jobs remain.

This node creates the final "apply" decisions.
Later modules (LLM reasoning, email reporting)
use this output.
"""

def decide_apply_node(state: AgentState) -> AgentState:
    decisions = []

    for job in state["jobs"]:
        decisions.append({
            "title": job["title"],
            "company": job["company"],
            "decision": "apply"
        })

    state["decisions"] = decisions
    return state


# -------------------------------------------------------------------
# REGISTER NODES IN GRAPH
# -------------------------------------------------------------------
"""
Nodes must be registered with the graph.
Each node is a named step.
"""

graph.add_node("discover_jobs", discover_jobs_node)
graph.add_node("check_masked", check_masked_node)
graph.add_node("check_role", check_role_node)
graph.add_node("check_experience", check_experience_node)
graph.add_node("decide_apply", decide_apply_node)


# -------------------------------------------------------------------
# DEFINE EXECUTION FLOW
# -------------------------------------------------------------------
"""
This is where LangGraph becomes powerful.

We explicitly define:
- where the agent starts
- the order of execution
- where it ends
"""

graph.set_entry_point("discover_jobs")
graph.add_edge("discover_jobs", "check_masked")
graph.add_edge("check_masked", "check_role")
graph.add_edge("check_role", "check_experience")
graph.add_edge("check_experience", "decide_apply")
graph.add_edge("decide_apply", END)


# -------------------------------------------------------------------
# COMPILE GRAPH
# -------------------------------------------------------------------
"""
Compilation converts the graph definition
into an executable agent.
"""

app = graph.compile()


# -------------------------------------------------------------------
# RUN THE AGENT
# -------------------------------------------------------------------
"""
This is the ONLY thing you run from terminal:

python agent_graph.py

Everything else happens automatically.
"""

if __name__ == "__main__":
    initial_state = {
        "jobs": [],
        "skipped_jobs": [],
        "roles_targeted": ["ai engineer", "machine learning engineer"],
        "max_experience_allowed": 5,
        "decisions": []
    }

    result = app.invoke(initial_state)
    print(result)


# job_discovery.py
This is the file that touches the outside world.
Everything before was logic. This is data acquisition.

In [ ]:
"""
JOB_DISCOVERY.PY — JOB FETCHING & SCRAPING LAYER
================================================

Purpose of this file:
- Go outside the agent
- Fetch real job postings
- Return structured data

IMPORTANT DESIGN IDEA:
----------------------
The agent itself does NOT scrape the web.
This file does.

Why?
- Separation of concerns
- Easier debugging
- Easier replacement later (API instead of scraping)

Agent_graph.py does NOT care HOW jobs are fetched.
It only cares about the RESULT.
"""

from playwright.sync_api import sync_playwright
from typing import List, Dict, Any


# -------------------------------------------------------------------
# MAIN FUNCTION EXPOSED TO THE AGENT
# -------------------------------------------------------------------
"""
This is the ONLY function agent_graph.py calls from this file.

Rule:
- If a function is not used by the agent, it should not be exposed.
"""

def get_jobs() -> Dict[str, List[Dict[str, Any]]]:
    """
    Returns:
    {
        "jobs":        list of valid job postings,
        "skipped_jobs": list of skipped jobs with reasons
    }
    """

    jobs = []
    skipped_jobs = []

    # Playwright is used instead of requests/bs4 because:
    # - LinkedIn uses heavy JavaScript
    # - Pages load dynamically
    # - Normal scraping fails
    with sync_playwright() as p:
        browser = p.chromium.launch(headless=True)
        page = browser.new_page()

        # NOTE:
        # This URL is usually a LinkedIn job search URL
        # In practice, it can change often
        search_url = "https://www.linkedin.com/jobs/search/?keywords=ai%20engineer"

        page.goto(search_url, timeout=60000)

        # Wait until job cards load
        page.wait_for_selector("ul.jobs-search__results-list")

        job_cards = page.query_selector_all("li.jobs-search-results__list-item")

        for card in job_cards:
            try:
                title_el = card.query_selector("h3")
                company_el = card.query_selector("h4")

                # Defensive programming:
                # Pages change, elements disappear
                if not title_el or not company_el:
                    skipped_jobs.append({
                        "reason": "missing_title_or_company"
                    })
                    continue

                title = title_el.inner_text().strip()
                company = company_el.inner_text().strip()

                # This project intentionally keeps parsing minimal.
                # More parsing = more fragility.
                jobs.append({
                    "title": title,
                    "company": company,

                    # Placeholder for later reasoning
                    # Some jobs include experience, some don't
                    "required_experience": None
                })

            except Exception as e:
                # If ONE job card fails, we do NOT kill the run
                skipped_jobs.append({
                    "reason": "exception_during_scrape",
                    "error": str(e)
                })

        browser.close()

    # IMPORTANT:
    # We always return the SAME STRUCTURE.
    # LangGraph nodes depend on predictable state shape.
    return {
        "jobs": jobs,
        "skipped_jobs": skipped_jobs
    }

Key mental model (important, read this)

job_discovery.py is NOT an agent

It has no reasoning

It has no LLM

It has no decisions

It is just:

“Go get raw jobs and don’t crash.”

If tomorrow you:

replace LinkedIn with an API

use RSS feeds

use Indeed

use a paid data source

👉 Only this file changes
The agent graph stays intact.

# batch_reasoning.py

This is where the project crosses the line from automation to intelligence.
Up to now, everything was rules.
Here is where LLMs start thinking.

In [ ]:
"""
BATCH_REASONING.PY — APPLYING LLM REASONING AT SCALE
===================================================

Purpose of this file:
- Take filtered jobs from the agent
- Run LLM reasoning on EACH job
- Attach human-readable explanations

Why this file exists:
--------------------
We separate reasoning from orchestration.

agent_graph.py:
- controls flow
- controls order
- controls state

batch_reasoning.py:
- focuses ONLY on "thinking"

This keeps the agent modular and testable.
"""

import json
from typing import List, Dict, Any

# This is the LLM-powered reasoning function
from llm_reasoner import explain_job


# -------------------------------------------------------------------
# MAIN FUNCTION USED BY THE AGENT
# -------------------------------------------------------------------
"""
This function is used after job filtering is complete.

Input:
- jobs: list of clean jobs
- user_profile: user preferences

Output:
- enriched job decisions with explanations
"""

def evaluate_jobs(
    jobs: List[Dict[str, Any]],
    user_profile: Dict[str, Any]
) -> List[Dict[str, Any]]:
    evaluated_jobs = []

    for job in jobs:
        try:
            # Ask the LLM to reason about THIS job
            llm_result = explain_job(job, user_profile)

            # Combine original job data with LLM reasoning
            evaluated_jobs.append({
                "title": job.get("title"),
                "company": job.get("company"),
                "required_experience": job.get("required_experience"),

                # LLM outputs
                "decision": llm_result.get("decision"),
                "reason": llm_result.get("reason"),
                "focus_keywords": llm_result.get("focus_keywords")
            })

        except Exception as e:
            # IMPORTANT DESIGN CHOICE:
            # One failed LLM call should NOT kill the whole run.
            evaluated_jobs.append({
                "title": job.get("title"),
                "company": job.get("company"),
                "error": str(e)
            })

    return evaluated_jobs


# -------------------------------------------------------------------
# OPTIONAL STANDALONE TEST RUN
# -------------------------------------------------------------------
"""
This block is ONLY for debugging.

You do NOT run this in production.
You do NOT call this from agent_graph.py.

It exists so you can test LLM reasoning
WITHOUT running the entire agent.
"""

if __name__ == "__main__":
    from job_discovery import get_jobs

    with open("user_profile.json", "r") as f:
        user_profile = json.load(f)

    jobs_data = get_jobs()
    jobs = jobs_data["jobs"]

    results = evaluate_jobs(jobs, user_profile)

    print(json.dumps(results, indent=2))


Key mental model (very important)

This file does NOT decide workflow

This file does NOT scrape

This file does NOT send emails

It only answers one question:

“Given this job and this user, what does an LLM think?”

That’s it.

Why this separation matters (real engineering reason)

If tomorrow you want to:

change the LLM

change the prompt

add scoring

add ranking

add feedback loops

👉 You only touch:

llm_reasoner.py

batch_reasoning.py

The agent graph stays untouched.

# llm_reasoner.py

This file is the thinking engine.
Everything intelligent in this project lives here.

In [ ]:
"""
LLM_REASONER.PY — SINGLE-JOB LLM INTELLIGENCE
============================================

Purpose of this file:
- Ask an LLM to reason about ONE job
- Compare it against ONE user profile
- Return a structured decision + explanation

This file does NOT:
- scrape jobs
- loop over multiple jobs
- send emails
- control workflow

It answers only one question:
"Should this user apply for THIS job, and why?"
"""

import os
import json

# LangChain wrapper for OpenAI-compatible APIs
from langchain_openai import ChatOpenAI

# Used to load environment variables from .env
from dotenv import load_dotenv


# -------------------------------------------------------------------
# LOAD ENVIRONMENT VARIABLES
# -------------------------------------------------------------------
"""
This allows us to keep API keys out of the code.

.env file contains:
OPENROUTER_API_KEY=xxxxxxxx
"""

load_dotenv()


# -------------------------------------------------------------------
# CREATE LLM CLIENT
# -------------------------------------------------------------------
"""
We use OpenRouter instead of OpenAI directly.

Why OpenRouter?
- Cheaper
- Model flexibility
- OpenAI-compatible API

Even though the class is ChatOpenAI,
it works with OpenRouter because the API is compatible.
"""

llm = ChatOpenAI(
    model="openai/gpt-4o-mini",  # lightweight, fast, cheap
    temperature=0.2,             # low randomness = more consistent reasoning
    api_key=os.getenv("OPENROUTER_API_KEY"),
    base_url="https://openrouter.ai/api/v1"
)


# -------------------------------------------------------------------
# CORE REASONING FUNCTION
# -------------------------------------------------------------------
"""
This is the ONLY function batch_reasoning.py calls.

Input:
- job: one job posting
- user_profile: resume + preferences

Output:
A dictionary with:
- decision: apply / skip
- reason: explanation in plain English
- focus_keywords: what the user should improve
"""

def explain_job(job: dict, user_profile: dict) -> dict:
    """
    Ask the LLM to reason like a career assistant.
    """

    # Build a clear, explicit prompt
    prompt = f"""
You are an AI career assistant.

User profile:
- Target roles: {user_profile.get("roles_targeted")}
- Experience: {user_profile.get("experience_years")} years
- Max allowed experience: {user_profile.get("max_experience_allowed")} years

Job posting:
- Title: {job.get("title")}
- Company: {job.get("company")}
- Required experience: {job.get("required_experience")}

Your task:
1. Decide if the user should apply ("apply" or "skip").
2. Explain the decision clearly.
3. List missing or important keywords the user should improve.

Respond STRICTLY in valid JSON with this structure:
{{
  "decision": "apply or skip",
  "reason": "short explanation",
  "focus_keywords": ["keyword1", "keyword2"]
}}
"""

    # Send prompt to the LLM
    response = llm.invoke(prompt)

    # LLMs return text, not Python objects
    raw_output = response.content.strip()

    # Defensive parsing:
    # LLMs sometimes return text before/after JSON
    try:
        result = json.loads(raw_output)
    except json.JSONDecodeError:
        # If parsing fails, return a safe fallback
        result = {
            "decision": "skip",
            "reason": "LLM output could not be parsed safely.",
            "focus_keywords": []
        }

    return result

Key mental model (important)

This file reasons about ONE job only

It has no loops

It has no workflow

It has no side effects

That makes it:

testable

replaceable

safe

If tomorrow you want:

better prompts

scoring instead of apply/skip

multiple reasoning passes

different models

👉 You change only this file.

# email_report.py

This file is presentation logic.
The agent has already decided.
The LLM has already reasoned.
Now we convert results into something a human can read.

In [ ]:
"""
EMAIL_REPORT.PY — HUMAN-READABLE SUMMARY GENERATION
==================================================

Purpose of this file:
- Take final agent decisions
- Convert them into a clean email-friendly report
- No scraping, no LLM calls, no workflow logic

Think of this file as:
"How do I explain the agent’s output to a human?"
"""

from typing import List, Dict


# -------------------------------------------------------------------
# MAIN REPORT GENERATION FUNCTION
# -------------------------------------------------------------------
"""
This function prepares TEXT.
It does NOT send the email.
It only formats content.

Input:
- evaluated_jobs: output from batch_reasoning.py

Output:
- subject: email subject line
- body: email body text
"""

def generate_email_report(evaluated_jobs: List[Dict]) -> Dict[str, str]:
    """
    Build a readable email summary from agent decisions.
    """

    # If no jobs were found or all were skipped
    if not evaluated_jobs:
        return {
            "subject": "Daily Job Agent Report — No Matching Jobs",
            "body": (
                "Hello,\n\n"
                "Your AI job agent ran successfully today.\n"
                "No suitable jobs were found based on your profile.\n\n"
                "This is a confirmation email — the system is working.\n\n"
                "Regards,\n"
                "Your AI Job Agent"
            )
        }

    # Email header
    body_lines = [
        "Hello,\n",
        "Your AI job agent analyzed today's job postings.\n",
        "Here are the most relevant opportunities:\n"
    ]

    # Loop through evaluated jobs
    for idx, job in enumerate(evaluated_jobs, start=1):
        body_lines.append(
            f"{idx}. {job.get('title')} at {job.get('company')}\n"
        )
        body_lines.append(
            f"   Decision: {job.get('decision')}\n"
        )
        body_lines.append(
            f"   Reason: {job.get('reason')}\n"
        )

        keywords = job.get("focus_keywords", [])
        if keywords:
            body_lines.append(
                f"   Suggested skills to improve: {', '.join(keywords)}\n"
            )

        body_lines.append("\n")

    body_lines.append(
        "This report was generated automatically by your AI agent.\n\n"
        "Regards,\n"
        "Your AI Job Agent"
    )

    return {
        "subject": "Daily Job Agent Report — Matching Opportunities Found",
        "body": "".join(body_lines)
    }


Key mental model (important)

This file does not think

This file does not decide

This file does not automate

It only answers:

“How should this look to a human?”

This separation is critical:

You can redesign emails without touching the agent

You can add HTML later

You can add Slack / WhatsApp later

Why this file matters in real systems

Most AI projects stop at:

“Here’s some JSON.”

Real systems need:

summaries

confirmations

explanations

trust

This file provides that bridge.

# email_sender.py

This is the delivery layer.
Nothing intelligent happens here.
It simply sends what was already prepared.

In [ ]:
"""
EMAIL_SENDER.PY — EMAIL DELIVERY LAYER
=====================================

Purpose of this file:
- Take a prepared email report
- Send it to the user via Gmail SMTP

This file does NOT:
- decide anything
- reason about jobs
- format content

It answers ONE question:
"How do we deliver the result to the user?"
"""

import os
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

from dotenv import load_dotenv


# -------------------------------------------------------------------
# LOAD ENVIRONMENT VARIABLES
# -------------------------------------------------------------------
"""
Email credentials are stored in .env for security.

.env contains:
- SENDER_EMAIL
- SENDER_PASSWORD (Gmail App Password)

NEVER hardcode credentials.
NEVER commit .env to GitHub.
"""

load_dotenv()


# -------------------------------------------------------------------
# MAIN EMAIL SENDING FUNCTION
# -------------------------------------------------------------------
"""
This function is called by the agent after the report is generated.

Input:
- subject: email subject
- body: email body text
- receiver_email: where to send the email
"""

def send_email(subject: str, body: str, receiver_email: str) -> None:
    sender_email = os.getenv("SENDER_EMAIL")
    sender_password = os.getenv("SENDER_PASSWORD")

    # Basic validation
    if not sender_email or not sender_password:
        raise ValueError("Email credentials not found in environment variables.")

    # Create email object
    message = MIMEMultipart()
    message["From"] = sender_email
    message["To"] = receiver_email
    message["Subject"] = subject

    # Attach email body as plain text
    message.attach(MIMEText(body, "plain"))

    # Gmail SMTP configuration
    smtp_server = "smtp.gmail.com"
    smtp_port = 587

    try:
        # Connect to Gmail SMTP
        server = smtplib.SMTP(smtp_server, smtp_port)
        server.starttls()  # Secure connection
        server.login(sender_email, sender_password)

        # Send email
        server.send_message(message)
        server.quit()

        print("Email sent successfully.")

    except Exception as e:
        print("Failed to send email.")
        print(str(e))


Mental model (very important)

This file is replaceable

You could swap Gmail for:

AWS SES

SendGrid

Slack

WhatsApp

Telegram

And the rest of the project would remain unchanged.

In [1]:
# python agent_graph.py    (terminal)

# Big picture (tie everything together)

job_discovery.py → gets raw jobs

agent_graph.py → controls the agent workflow (LangGraph)

batch_reasoning.py → applies LLM reasoning at scale

llm_reasoner.py → single-job intelligence

email_report.py → human-readable summary

email_sender.py → delivery